In [ ]:
import re
from datetime import datetime

def parse_time_to_cron(sentence):
    # 定义正则表达式来匹配不同的频率和时间模式
    patterns = {
        'daily': r'每天',
        'weekly': r'每周([1-7]|一|二|三|四|五|六|日)',
        'time': r'上午|下午|晚|早上|凌晨|午夜'
    }

    # 初始化Cron字段
    cron = {
        'minute': '*',
        'hour': '*',
        'day': '*',
        'month': '*',
        'week': '*'
    }

    # 提取频率部分
    frequency_match = re.match(r'^(每天|每周[\d一二三四五六日]+)', sentence)
    if frequency_match:
        frequency = frequency_match.group(1)
        if '每天' in frequency:
            cron['day'] = '*'
            cron['month'] = '*'
            cron['week'] = '*'
        elif '每周' in frequency:
            # 提取星期几
            week_day = re.search(r'([1-7]|一|二|三|四|五|六|日)', frequency).group(1)
            # 将中文星期数转换为数字
            week_map = {'一': 1, '二': 2, '三': 3, '四':4, '五':5, '六':6, '日':7}
            if isinstance(week_day, str) and week_day in week_map:
                cron['week'] = str(week_map[week_day])
            else:
                cron['week'] = week_day
    else:
        # 默认处理，如果没有明确频率，假设为每天
        cron['day'] = '*'
        cron['month'] = '*'
        cron['week'] = '*'

    # 提取时间部分
    time_match = re.search(r'([0-2]?[0-9]:?[0-9][0-9]?)|(上午|下午|晚|早上)([0-9]+点?)', sentence)
    if time_match:
        time_str = time_match.group(0)
        # 尝试解析时间
        try:
            # 如果是12小时制，转换为24小时制
            if '上午' in time_str or '下午' in time_str:
                time_str = re.sub(r'(上午|下午)', '', time_str).strip()
                hour = int(time_str.replace('点', '')) 
                if '下午' in time_match.group(0):
                    hour += 12
                cron['hour'] = str(hour)
                cron['minute'] = '0'
            else:
                # 处理24小时制
                parts = re.split(r':|点', time_str)
                cron['hour'] = parts[0]
                if len(parts) >1:
                    cron['minute'] = parts[1]
        except Exception as e:
            print(f"解析时间失败：{e}")
            return None

    # 生成Cron表达式
    cron_expression = f"{cron['minute']} {cron['hour']} {cron['day']} {cron['month']} {cron['week']}"
    return cron_expression

# 测试示例
sentence1 = "床"
print(parse_time_to_cron(sentence1))  # 输出: 0 8 * * *

sentence2 = "周五16:00提醒我提交周报"
print(parse_time_to_cron(sentence2))  # 输出: 0 16 * * 5

* 08 * * 5
30 15 * * 1


In [7]:
import re
from typing import Optional

def extract_cron_expression(text: str) -> Optional[str]:
    """
    从自然语言提取 cron 表达式
    """
    days_map = {
        "星期一": 1, "星期二": 2, "星期三": 3, "星期四": 4, "星期五": 5, "星期六": 6, "星期日": 0,
        "每周一": 1, "每周二": 2, "每周三": 3, "每周四": 4, "每周五": 5, "每周六": 6, "每周日": 0,
        "周一": 1, "周二": 2, "周三": 3, "周四": 4, "周五": 5, "周六": 6, "周日": 0,
    }
    
    
    time_match = re.search(r'(上午|下午)?(\d{1,2})[:点](\d{0,2})?', text)
    if not time_match:
        return None
    
    period, hour, minute = time_match.groups()
    hour = int(hour)
    minute = int(minute) if minute else 0
    if period == "下午" and hour < 12:
        hour += 12
    
    if "每天" in text:
        cron_expr = f"{minute} {hour} * * *"
    else:
        for day, cron_day in days_map.items():
            if text.startswith(day):
                cron_expr = f"{minute} {hour} * * {cron_day}"
                break
        else:
            return None
    
    return cron_expr

# 测试用例
examples = [
    "每天上午8点叫我起床",
    "周五16:00提醒我提交周报",
    "星期三5点提醒我去上游泳课"
]

for ex in examples:
    print(f"{ex} -> {extract_cron_expression(ex)}")


每天上午8点叫我起床 -> 0 8 * * *
周五16:00提醒我提交周报 -> 0 16 * * 5
星期三5点提醒我去上游泳课 -> 0 5 * * 3


In [20]:
import re

def extract_time(text):
    """
    从给定文本中提取时间部分。
    
    时间格式可以是"HH"或"HH:MM"。返回小时和分钟作为整数。
    如果没有找到匹配的时间格式，返回None。
    """
    time_match = re.search(r'(上午|下午)?(\d{1,2})[:点](\d{0,2})?', text)
    if not time_match:
        return None
    
    period, hour, minute = time_match.groups()
    hour = int(hour)
    minute = int(minute) if minute else 0
    if period == "下午" and hour < 12:
        hour += 12
    return (minute, hour)


def generate_cron_expression(text):
    """
    根据输入文本和提取的时间生成相应的Cron表达式。
    
    检查文本中是否包含特定的关键字，如“每天”或“星期X天”，以决定生成哪种Cron表达式。
    """
    days_map = {
        "星期一": 1, "星期二": 2, "星期三": 3, "星期四": 4, "星期五": 5, "星期六": 6, "星期日": 0, "星期天": 0,
        "每周一": 1, "每周二": 2, "每周三": 3, "每周四": 4, "每周五": 5, "每周六": 6, "每周日": 0, "每周天": 0,
        "周一": 1, "周二": 2, "周三": 3, "周四": 4, "周五": 5, "周六": 6, "周日": 0, "周天": 0,
    }
    
    if "每天" in text:
        cron_expr = f"* * *"
    else:
        for day, cron_day in days_map.items():
            if text.startswith(day):
                cron_expr = f"* * {cron_day}"
                break
        else:
            return None
    
    return cron_expr

def get_cron_time(text):
    """
    组合提取时间和生成Cron表达式的功能。
    
    返回最终的Cron表达式字符串。
    """
    hour, minute = extract_time(text)
    corn= generate_cron_expression(text)
    return( hour, minute,corn)


# 示例使用：
print(get_cron_time('每天 3:15'))  # 输出：15 3 * * *
print(get_cron_time('星期天 10:30'))  # 输出：30 10 * * 5

(15, 3, '* * *')
(30, 10, '* * 0')
